In [4]:
# %%capture
# !pip install openai
# !pip install python-dotenv
# !pip install urllib3

In [7]:
import json
import os
from dotenv import load_dotenv
from openai import OpenAI
import time
import function_call_limit_functions as FC
from function_call_tools import tools, instruction1, instruction2
load_dotenv()

True

#### The class AssistantManager will store functions for creating assistants, creating threads for the assistants, adding message to the threads, running the assistants. Then we will wait for the the completions and submit the response back to the assistant again for generating a response.

In [8]:
class AssistantManager:
  def __init__(self,api_key:str,model:str="gpt-3.5-turbo-1106"):
    self.client = OpenAI(api_key=api_key)
    self.model = model
    self.assistant = None
    self.thread = None
    self.run = None
    self.file = None
    self.message = None

  def create_assistant(self,name, instructions, tools,id):
        assistant_file_path = './assistant.json'
        if os.path.exists(assistant_file_path):
          with open(assistant_file_path, 'r') as file:
            assistant_data = json.load(file)
            if f'{id}' in assistant_data:
              assistant_id = assistant_data[f'{id}']['assistant_id']
              self.assistant = self.client.beta.assistants.retrieve(assistant_id)
              file.close()
            else:
               self.assistant = self.client.beta.assistants.create(
                name = name,
                instructions = instructions,
                tools = tools,
                model = self.model
                )
               assistant_data[f'{id}'] = {'assistant_id': self.assistant.id}
               with open(assistant_file_path, 'w') as file:
                json.dump(assistant_data, file)
                file.close()
               
        else:
            self.assistant = self.client.beta.assistants.create(
                name = name,
                instructions = instructions,
                tools = tools,
                model = self.model
            )
            with open(assistant_file_path, 'w') as file:
              json.dump({f'{id}':{'assistant_id': self.assistant.id}}, file)
              print("Created a new assistant and saved the ID.")
              file.close()

  def create_thread(self,id):
      thread_file_path = './assistant_thread.json'
      if os.path.exists(thread_file_path):
        with open(thread_file_path, 'r') as file:
          thread_data = json.load(file)
          if f'{id}' in thread_data:
            thread_id = thread_data[f'{id}']['thread_id']
            self.thread = self.client.beta.threads.retrieve(thread_id = thread_id)
            file.close()
          else:
             self.thread = self.client.beta.threads.create()
             thread_data[f'{id}'] = {'thread_id': self.thread.id}
             with open(thread_file_path, 'w') as file:
              json.dump(thread_data, file)
              file.close()
      else:
        self.thread = self.client.beta.threads.create()
        with open(thread_file_path, 'w') as file:
              json.dump({f'{id}':{'thread_id': self.thread.id}}, file)
              print("Created a new thread and saved the ID.")
              file.close()

  def add_message_to_thread(self,role,content):
    self.message = self.client.beta.threads.messages.create(
      thread_id = self.thread.id,
      role = role,
      content = content
    )

  def run_assistant(self,instructions):
    self.run = self.client.beta.threads.runs.create(
      thread_id = self.thread.id,
      assistant_id=self.assistant.id,
      instructions = instructions
    )

  def process_messages(self):
    messages = self.client.beta.threads.messages.list(thread_id = self.thread.id)

    msg = messages.data[0]
    role = msg.role
    content = msg.content[0].text.value
    #print(f"{role.capitalize}:{content}")
    return content

  def wait_for_completion(self):
    while True:
      time.sleep(5)
      run_status = self.client.beta.threads.runs.retrieve(
        thread_id = self.thread.id,
        run_id = self.run.id
      )
      
      print(run_status.status)
      if run_status.status == 'completed':
        return(self.process_messages())
        break
      elif run_status.status == 'requires_action':
        print("Function Calling ...")
        self.call_required_functions(run_status.required_action.submit_tool_outputs.model_dump())
      elif run_status.status=="failed":
        print(run_status.last_error)
        return ("The assistant got rate limited, try again after some time or refresh the page and try again")
        break
      else:
        print("Waiting for the Assistant to process..") 
  
  def call_required_functions(self, required_actions):
    tool_output = []

    for action in required_actions["tool_calls"]:
      func_name = action['function']['name']
      arguments = json.loads(action['function']['arguments'])
      print(arguments)
      print(func_name)
      func_name = eval("FC."+func_name)
      output = func_name(**arguments)
      tool_output.append(
        {
          "tool_call_id":action['id'],
          "output": str(output)
        }
      )
    
    print("Submitting outputs back to the Assistants...")
    self.client.beta.threads.runs.submit_tool_outputs(
      thread_id = self.thread.id,
      run_id = self.run.id,
      tool_outputs=tool_output
    )

In [9]:
api_key = os.getenv("Open_AI_key")
manager = AssistantManager(api_key)

manager.create_assistant(
    name = "Function_limit",
    instructions=instruction1,
    tools = tools,
    id = "673224"
)

Created a new assistant and saved the ID.


In [14]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="sentiment analysis for the text 'Yesterday was a wonderful day! I spent the morning hiking through the forest, enjoying the crisp air and vibrant colors of autumn. In the afternoon, I met up with some friends for a delicious picnic by the lake. The laughter and camaraderie lifted my spirits, and I felt truly grateful for the simple joys of life. However, when I got home, I received some disappointing news that dampened my mood. Despite that, I'm choosing to focus on the positive moments and carry that happiness into today.'")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'text': "Yesterday was a wonderful day! I spent the morning hiking through the forest, enjoying the crisp air and vibrant colors of autumn. In the afternoon, I met up with some friends for a delicious picnic by the lake. The laughter and camaraderie lifted my spirits, and I felt truly grateful for the simple joys of life. However, when I got home, I received some disappointing news that dampened my mood. Despite that, I'm choosing to focus on the positive moments and carry that happiness into today."}
analyze_sentiment
Submitting outputs back to the Assistants...
completed
The sentiment analysis of the given text indicates a "positive" sentiment.


In [15]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="check the email rishavmitra3")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'email': 'rishavmitra3'}
is_valid_email
Submitting outputs back to the Assistants...
completed
The email pattern "rishavmitra3" is not valid. Please make sure to include the appropriate email format (e.g., example@gmail.com).


In [16]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="check the password rishavmitra3")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'password': 'rishavmitra3'}
password_strength
Submitting outputs back to the Assistants...
completed
The password "rishavmitra3" has been determined to have "Strong" strength.


In [17]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="calculate the entropy for 30.12 Joules and 100.482 Kelvin")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'temperature': 100.482, 'delta_energy': 30.12}
calculate_entropy
Submitting outputs back to the Assistants...
completed
The entropy calculated based on the given temperature of 100.482 Kelvin and delta energy of 30.12 Joules is approximately 0.2998 Joules/Kelvin.


In [19]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="120 meters and 50 Hz")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'wavelength': 120, 'frequency': 50}
wave_speed
Submitting outputs back to the Assistants...
completed
The wave speed calculated with a wavelength of 120 meters and a frequency of 50 Hz is 6000 m/s.


In [20]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="Calculate the projectile motion for 100degrees and 30 m/s")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'v0': 30, 'theta': 100}
projectile_motion_range
Submitting outputs back to the Assistants...
completed
The calculated range for the projectile motion with an initial velocity of 30 m/s and a launch angle of 100 degrees is approximately -31.41 meters. It's important to note that this negative value implies the projectile will land behind the launch point, likely due to the launch angle and initial velocity combination.


In [23]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="Convert 32C")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'celsius': 32}
celsius_to_fahrenheit
Submitting outputs back to the Assistants...
completed
32°C is equivalent to 89.6°F.


In [24]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="Find all even square number till 50")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'n': 50}
even_squares
Submitting outputs back to the Assistants...
completed
The even square numbers up to 50 are: 0, 4, 16, 36, 64, 100, 144, 196, 256, 324, 400, 484, 576, 676, 784, 900, 1024, 1156, 1296, 1444, 1600, 1764, 1936, 2116, and 2304.


In [26]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="10001101 to decimal")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'binary': '10001101'}
binary_to_decimal
Submitting outputs back to the Assistants...
completed
The decimal equivalent of the binary number 10001101 is 141.


In [27]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="What is the weather like in 425 stewart dr mountian view ca 94043?")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'address': '425 Stewart Dr, Mountain View, CA 94043'}
weather_forecast
Submitting outputs back to the Assistants...
completed
The weather in Mountain View, CA (425 Stewart Dr, 94043) is currently characterized by scattered clouds with a temperature of approximately 280.74 K (7.59°C), a minimum temperature of 279.46 K (6.31°C), and a maximum temperature of 282.19K (9.04°C). The wind speed is about 3.09 m/s coming from 160 degrees and the humidity level is at 91%.


In [28]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="Calculate the mean and standard deviation of 1,3,4,62,72,1,20")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'numbers': [1, 3, 4, 62, 72, 1, 20]}
calculate_mean_std
Submitting outputs back to the Assistants...
completed
For the given numbers 1, 3, 4, 62, 72, 1, and 20, the mean is approximately 23.29 and the standard deviation is approximately 28.43.


In [29]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="Solve the equation 6x^2 + 8x + 21")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'a': 6, 'b': 8, 'c': 21}
solve_quadratic_equation
Submitting outputs back to the Assistants...
completed
The roots of the quadratic equation 6x^2 + 8x + 21 are complex and given by: 
-0.67 + 1.75i (i is the imaginary unit) and -0.67 - 1.75i.


In [31]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="Calculate the BMI for 70Kg and 1.532m")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'weight_kg': 70, 'height_m': 1.532}
calculate_bmi
Submitting outputs back to the Assistants...
completed
The BMI calculated for a person weighing 70Kg and having a height of 1.532m is approximately 29.83.


In [32]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="Calculate distance between (50,21) and (30,75.5)")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'x1': 50, 'y1': 21, 'x2': 30, 'y2': 75.5}
calculate_distance
Submitting outputs back to the Assistants...
completed
The distance between the points (50, 21) and (30, 75.5) is approximately 58.05 units.


In [34]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="Calculate my monthly payment for principal amount of $2000 at an interest of 20% for 1 year")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'principal': 2000, 'annual_interest_rate': 20, 'years': 1}
calculate_monthly_payment
Submitting outputs back to the Assistants...
completed
The monthly payment for a principal amount of $2000 at an interest rate of 20% for 1 year is approximately $185.27.
